In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time
import datetime
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

from input_processor import InputProcessor, load_embeddings, batch_iter, train_batch_iter, val_batch_iter
from input_processor import test_batch_iter
from siamese_network import SiameseLSTM
from train import prepare_data, train_network
from predict import generate_prediction
from random import random

from keras.preprocessing import sequence, text
from keras.utils import np_utils
import pickle

Using TensorFlow backend.


In [2]:
n_layers = 3
dense_units=1024
max_doc_length = 50
embedding_dim = 300
hidden_units = 256
l2_reg_lambda = 0.1
batch_size = 512+256
num_epochs = 15
evaluate_every = 1
dropout_keep_prob = 0.65
lr = 1e-3
multiply = True
basic_lstm = False
ignore_one_in_every = 3

vocab_size = 200000

prepare = False
embedding_path = '/opt/datasets/glove/glove.840B.300d.txt'

In [3]:
if prepare:
    prepare_data(embedding_path, vocab_size, max_doc_length)

In [7]:
tot_loss = 0
(checkpoint_file, loss) = train_network(n_layers=n_layers,
                                    dense_units=dense_units,
                                    max_doc_length=max_doc_length,
                                    embedding_dim=embedding_dim, hidden_units=hidden_units,
                                    l2_reg_lambda=l2_reg_lambda,
                                    batch_size=batch_size, num_epochs=num_epochs,
                                    evaluate_every=evaluate_every,
                                    dropout_keep_prob=dropout_keep_prob, lr=lr,
                                    multiply=multiply, basic_lstm=basic_lstm,
                                    ignore_one_in_every=ignore_one_in_every,
                                    early_stopping=False)

Text file name:  layers_3-dense_units_1024-hidden_256-l2_0.1-dropout_0.65-multiplyTrue-basiclstm_False-ignore_3-2017-04-21T17:31:16.537266.txt
starting graph def
started session
Done adding...
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Tensor("embedding_placeholder:0", shape=(219552, 300), dtype=float32, device=/device:CPU:0)
Tensor("W/read:0", shape=(219552, 300), dtype=float32, device=/device:CPU:0)
Tensor("Assign:0", shape=(219552, 300), dtype=float32_ref, device=/device:CPU:0)
Tensor("input_x1:0", shape=(?, 50), dtype=int32, device=/device:GPU:0)
Tensor("input_x2:0", shape=(?, 50), dtype=int32, device=/device:GPU:0)
Tensor("embed1:0", shape=(?, 50, 300), dtype=float32, device=/device:CPU:0)
Tensor("embed2:0", shape=(?, 50, 300), dtype=float32, device=/device:CPU:0)
Tensor("input_y:0", shape=(?,), dtype=int32, device=/device:GPU:0)
Tensor("dropout_keep_prob:0", dtype=float32, device=/device:GPU:0)
Tensor("output/sentence_embedding:0", shape=(?, 512), dtype=float32, device=/device:GPU:0

0it [00:00, ?it/s]

Starting epoch: 0 at 2017-04-21T17:31:45.465971
Generating batches for data of shape (363861, 6).
num_batches_per_epoch:  474


AttributeError: 'numpy.ndarray' object has no attribute 'get_shape'

In [5]:
(checkpoint_file, loss)

# ('/opt/deeplearning/git/quorakaggle/runs/1492730085/checkpoints/model-5688',
#  54.222349926829338)

# ('/opt/deeplearning/git/quorakaggle/runs/1492755160/checkpoints/model-1896',
#  18.610518276691437)

('/opt/deeplearning/git/quorakaggle/runs/1492755160/checkpoints/model-1896',
 18.610518276691437)

In [6]:
old_checkpoint_file = '/home/sachint/work/kaggle/quora/runs/1492108813/checkpoints/model-25587'
old_checkpoint_file = '/home/sachint/work/kaggle/quora/runs/1492261127/checkpoints/model-48331'
old_checkpoint_file = '/home/sachint/work/kaggle/quora/runs/1492373117/checkpoints/model-17058'
sub = generate_prediction(checkpoint_file=old_checkpoint_file, batch_size=128, 
                          batch_generator=test_batch_iter, has_labels=False)
a = 0.165 / 0.37
b = (1 - 0.165) / (1 - 0.37)
sub['is_duplicate'] = sub['is_duplicate'].apply(lambda x: a * x / (a * x + b * (1 - x)))
sub.to_csv('submission.csv', index=False)

Loading TF checkpoint from: /home/sachint/work/kaggle/quora/runs/1492373117/checkpoints/model-17058
Instructions for updating:
Use `tf.global_variables_initializer` instead.


0it [00:00, ?it/s]

Generating batches for data of shape (2345796, 5).


184it [1:07:09, 16.83s/it]


In [13]:
l1 = [[1, 2, 3, 4], [1, 1, 1, 1]]
l2 = [[2, 2, 2, 2], [3, 3, 3, 3]]
np.reshape([min(len(x), 3) for x in l], [-1])[0]
with tf.Graph().as_default():
    sess = tf.Session()
    print("started session")
    with sess.as_default():
        res = sess.run(tf.concat(l1, axis=0))
        print(res)

started session
[1 2 3 4 1 1 1 1]


In [14]:
print(tf.__version__)

1.0.1


In [10]:
from input_processor import InputProcessor, load_embeddings, batch_iter, train_batch_iter, val_batch_iter
batches = train_batch_iter(5)

for b in batches:
    print(b[1][0])
    print(b[1][1])
    print(b[1][2])
    print(b[1][3])
    
    break

Generating batches for data of shape (363861, 6).
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0   22   19 1076
    9 1581  171 3039    1]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0   22   19  239   14  579   12  412    9 1581  208   25
  317  563    6  133    1]
8
16


In [29]:
sub1 = pd.read_csv("submission.csv")
sub2 = pd.read_csv("submission2.csv")
sub3 = pd.read_csv("submission_good_calib.csv")
sub3.rename(columns={'test_id': 'test_id', 'is_duplicate': 'is_duplicate_z'}, inplace=True)

In [30]:
tmp = pd.merge(sub1, sub2, on='test_id')
merged = pd.merge(tmp, sub3, on='test_id')

In [31]:
merged

,test_id,is_duplicate_x,is_duplicate_y,is_duplicate_z
0,0,0.047425,0.087695,0.077585
1,1,0.060019,0.072718,0.044134
2,2,0.088643,0.114912,0.112221
3,3,0.006418,0.160392,0.122988
4,4,0.101623,0.071513,0.000482
5,5,0.002122,0.000759,0.000226
6,6,0.810365,0.637107,0.580960
7,7,0.988649,0.833306,0.112210
8,8,0.658144,0.621951,0.349855
9,9,0.048011,0.206985,0.092779


In [32]:
merged['is_duplicate'] = merged['is_duplicate_x']/3 + merged['is_duplicate_y']/3 + merged['is_duplicate_z']/3

In [33]:
merged[['test_id','is_duplicate']].to_csv('submission_merged_3.csv', index=False)

In [7]:
sub_last = pd.read_csv('submissions/submission_merged_4.csv')
sub = pd.read_csv('submission.csv')

In [8]:
sub_last

,test_id,is_duplicate
0,0,0.084538
1,1,0.065079
2,2,0.090168
3,3,0.086720
4,4,0.060228
5,5,0.004187
6,6,0.672552
7,7,0.708359
8,8,0.479139
9,9,0.087685


In [9]:
sub = pd.merge(sub, sub_last, on="test_id")
sub['is_duplicate'] = sub['is_duplicate_x']*1/2 + sub['is_duplicate_y']*1/2
sub[['test_id', 'is_duplicate']].to_csv('submission_merged_5.csv', index=False)